#EN3150 Assignment 03: Simple convolutional neural network to perform classification.



##1 .CNN for image classification

### Mount the google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load the Dataset

In [3]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # Progress bar to monitor the copying process

# Define the base directory and the new directories for train, validation, and test sets
base_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/'
train_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/train'
validation_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/validation'
test_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/test'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Lists to hold image data and labels
image_paths = []
labels = []

# Load images from each folder (H1, H2, H3, H5, H6)
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):
                # Construct full image path
                image_path = os.path.join(folder_path, filename)
                image_paths.append(image_path)
                labels.append(folder)  # Assign label based on folder name

# Convert labels to a NumPy array
labels = np.array(labels)

### Split the Dataset

In [5]:
# Split data: 60% train, 20% validation, 20% test
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=0.4, random_state=42, stratify=labels
)
validation_paths, test_paths, validation_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

# Print the sizes of the splits
print(f'Training set: {len(train_paths)} images')
print(f'Validation set: {len(validation_paths)} images')
print(f'Test set: {len(test_paths)} images')

# Function to copy images to the relevant directories (train, validation, test)
def copy_images_to_directories(image_paths, labels, split_dir):
    # Create subdirectories for each class in the split directory (train, validation, test)
    for class_name in np.unique(labels):
        os.makedirs(os.path.join(split_dir, class_name), exist_ok=True)

    # Use tqdm to show progress while copying files
    for image_path, label in tqdm(zip(image_paths, labels), total=len(image_paths)):
        # Determine the class directory for the label
        class_dir = os.path.join(split_dir, label)
        # Copy the image to the appropriate class directory
        shutil.copy(image_path, class_dir)

# Copy images to the train, validation, and test directories
print("Copying images to the train directory...")
copy_images_to_directories(train_paths, train_labels, train_dir)

print("Copying images to the validation directory...")
copy_images_to_directories(validation_paths, validation_labels, validation_dir)

print("Copying images to the test directory...")
copy_images_to_directories(test_paths, test_labels, test_dir)

print("Images have been copied to train, validation, and test directories.")


Training set: 5468 images
Validation set: 1823 images
Test set: 1823 images
Copying images to the train directory...


100%|██████████| 5468/5468 [14:18<00:00,  6.37it/s]


Copying images to the validation directory...


100%|██████████| 1823/1823 [04:19<00:00,  7.03it/s]


Copying images to the test directory...


100%|██████████| 1823/1823 [04:21<00:00,  6.98it/s]

Images have been copied to train, validation, and test directories.


### Build the CNN Model with modifications

In [4]:
from tensorflow.keras import layers, models

# Define the CNN architecture
def build_cnn(input_shape,x1, m1, x2, m2, x3, x4, m3, d, num_classes):
    model = models.Sequential(name="Custom_CNN_Model")

    # Use Input layer to define the input shape
    model.add(layers.Input(shape=input_shape, name='Input'))

    # First Convolutional Layer
    model.add(layers.Conv2D(x1, (m1, m1), activation='relu', name='Conv2D_1'))
    model.add(layers.MaxPooling2D((2, 2), name='MaxPooling2D_1'))

    # Second Convolutional Layer
    model.add(layers.Conv2D(x2, (m2, m2), activation='relu', name='Conv2D_2'))
    model.add(layers.MaxPooling2D((2, 2), name='MaxPooling2D_2'))

    # Third Convolutional Layer (newly added)
    model.add(layers.Conv2D(x4, (m3, m3), activation='relu', name='Conv2D_3'))
    model.add(layers.MaxPooling2D((2, 2), name='MaxPooling2D_3'))

    # Flatten the output
    model.add(layers.Flatten(name='Flatten'))

    # Fully Connected Layer
    model.add(layers.Dense(x3, activation='relu', name='Dense_1'))
    model.add(layers.Dropout(d, name='Dropout_1'))  # Dropout layer to reduce overfitting

    # Additional Fully Connected Layer
    model.add(layers.Dense(x4, activation='relu', name='Dense_2'))
    model.add(layers.Dropout(d, name='Dropout_2'))  # Additional Dropout layer

    # Output Layer
    model.add(layers.Dense(num_classes, activation='softmax', name='Output'))

    return model

# Build and compile the CNN model
input_shape = (224,224, 3)
cnn_model = build_cnn(input_shape,x1=32, m1=3, x2=64, m2=3, x3=128, x4=64, m3=3, d=0.5, num_classes=5)
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
cnn_model.summary()

Model: "Custom_CNN_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Conv2D_1 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ MaxPooling2D_1 (MaxPooling2D)        │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2D_2 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ MaxPooling2D_2 (MaxPooling2D)        │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2D_3 (Conv2D)                    │ (None, 52, 52, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ MaxPooling2D_3 (MaxPooling2D)        │ (None, 26, 26, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Flatten (Flatten)                    │ (None, 43264)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dense_1 (Dense)                      │ (None, 128)                 │       5,537,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Output (Dense)                       │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,602,821 (21.37 MB)

 Trainable params: 5,602,821 (21.37 MB)

 Non-trainable params: 0 (0.00 B)

### Train the Model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
img_size = (224, 224)  # Resize images to 150x150
batch_size = 32

# Create ImageDataGenerators for training, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalize images to [0, 1]
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use categorical since labels are one-hot encoded
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No shuffling for validation data
)

# Train the model
history = cnn_model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    verbose=1
)


Found 5468 images belonging to 5 classes.
Found 1823 images belonging to 5 classes.
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
